In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import joblib
from functools import reduce
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import MinMaxScaler

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
#from skmultilearn.model_selection import MultiLabelStratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [11]:
# breed
feat_breed = pd.read_pickle('../features/breed.pkl')
feat_breed_v2 = pd.read_pickle('../features/breed_v2.pkl')

feat_breed_top50 = pd.read_pickle('../features/breed_top50.pkl')
feat_breed_top50_v2 = pd.read_pickle('../features/breed_top50_v2.pkl')

feat_breed_group = pd.read_pickle('../features/breed_group.pkl')
feat_sub_breed = pd.read_pickle('../features/sub_breed.pkl')
feat_breed_type = pd.read_pickle('../features/breed_type.pkl')

feat_breed_pure_or_mix = pd.read_pickle('../features/breed_pure_or_mix.pkl')

# age
feat_age = pd.read_csv('../features/age_with_id.csv')

# sex
feat_sex = pd.read_csv('../features/one_hot_encoded_sex_with_id.csv')

# weight
feat_weight = pd.read_pickle('../features/weight.pkl')

# Climate
# feat_HotWheater = pd.read_csv('../features/one_hot_encoded_HotWheater_with_id.csv')
# feat_ModerateWheather = pd.read_csv('../features/one_hot_encoded_ModerateWheather_with_id.csv')
# feat_ColdWheater_with_id = pd.read_csv('../features/one_hot_encoded_ColdWheater_with_id.csv')
feat_cold_month = pd.read_pickle('../features/ColdMonths.pkl')
feat_hot_month = pd.read_pickle('../features/HotMonths.pkl')

# Diet
feat_diet = pd.read_pickle('../features/diet.pkl') # 5 columns

# Physical Activity
feat_pa_total_hours = pd.read_csv('../features/PhysicalActivity_total_hours.csv')
feat_pa_surface = pd.read_csv('../features/PhysicalActivity_surface.csv')
# feat_pa_wheather = pd.read_csv('../features/PhysicalActivity_wheather.csv')

# Owner Demographics
feat_od_income = pd.read_pickle('../features/od_income.pkl')
feat_od_education = pd.read_pickle('../features/od_education.pkl')

# Residentual
feat_prim_census_division = pd.read_pickle('../features/primary_residence_census_division.pkl')

# disease
feat_disease_input = pd.read_csv('../features/one_hot_encoded_disease_input.csv')
feat_disease_output_binary = pd.read_csv('../features/disease_output_binary.csv')
feat_disease_output = pd.read_csv('../features/disease_output.csv')

# age_condition
feat_age_condition = pd.read_pickle('../features/age_condition.pkl')
feat_age_condition_type = pd.read_pickle('../features/age_condition_type.pkl')


features_list = [

    # breed
    # feat_breed,
    # feat_breed_v2,
    # feat_breed_top50,
    feat_breed_top50_v2,
    feat_breed_group,
    # feat_sub_breed,
    # feat_breed_type,
    # feat_breed_pure_or_mix,

    # age
    # feat_age,   # 24881 dog_ids

    # sex
    # feat_sex,

    # weight
    # feat_weight,

    # diet
    # feat_diet,  # 33141 dog_ids for df_diet_consistency

    #feat_HotWheater,
    #feat_ModerateWheather,
    #feat_ColdWheater_with_id,

    # Climate
    # feat_pa_wheather,   # 26406 dog_ids
    # feat_hot_month,
    # feat_cold_month,

    # Physical Activity
    # feat_pa_total_hours,
    # feat_pa_surface,

    # Owner demographics
    # feat_od_income,    # 29096 dog_ids
    # feat_od_education,

    # Residentual
    # feat_prim_census_division,

    # disease
    # feat_disease_input,
    #feat_disease_output_binary,
    #feat_disease_output
]

feat_age_condition_type

,dog_id,age_diagnosis_years,condition_type_Bone/Orthopedic,condition_type_Brain/Neurologic,condition_type_Cardiac,condition_type_Ear/Nose/Throat,condition_type_Endocrine,condition_type_Eye,condition_type_Gastrointestinal,condition_type_Hematopoietic,...,condition_type_Kidney/Urinary,condition_type_Liver/Pancreas,condition_type_Mouth/Dental/Oral,condition_type_Other Congenital Disorder,condition_type_Reproductive,condition_type_Respiratory,condition_type_Skin,condition_type_Toxin Consumption,condition_type_Trauma,condition_type_cancer
0,100001,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,100001,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,100001,2,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,100001,3,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,100001,4,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910915,3373,10,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
910929,14856,10,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
910936,58327,6,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
910945,3417,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [12]:
# List of DataFrames to be merged
list_input_features = features_list  # Add more DataFrames as needed

# Merge DataFrames iteratively using reduce
input_features = reduce(lambda left, right: pd.merge(left, right, on='dog_id'), list_input_features)

# Merge the two dataframes based on dog_id
#data = pd.merge(feat_breed_top50_v2, feat_breed_group, on='dog_id')

# Merge with disease output feature
data = pd.merge(feat_age_condition_type, input_features, on='dog_id')
data

,dog_id,age_diagnosis_years,condition_type_Bone/Orthopedic,condition_type_Brain/Neurologic,condition_type_Cardiac,condition_type_Ear/Nose/Throat,condition_type_Endocrine,condition_type_Eye,condition_type_Gastrointestinal,condition_type_Hematopoietic,...,breed_group_Mixed Lab and Golden,breed_group_Mixed Large,breed_group_Mixed Medium,breed_group_Mixed Other,breed_group_Mixed Small,breed_group_Shepherd,breed_group_Spaniels,breed_group_Terriers,breed_group_Toy - Other,breed_group_Working dogs - Non-sport
0,100001,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,100001,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,100001,2,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,100001,3,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,100001,4,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306423,77040,3,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
306424,77040,4,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
306425,77040,5,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
306426,77040,6,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
data['dog_id'].value_counts()

83756    44
64803    43
36023    43
9104     40
56174    39
         ..
84375     1
84317     1
84290     1
98034     1
78302     1
Name: dog_id, Length: 26692, dtype: int64

In [14]:
# Check for NaN values
rows_with_nan = data[data.isna().any(axis=1)]
columns_with_nan = data.columns[data.isna().any()].tolist()
# Display the rows with NaN values
rows_with_nan[columns_with_nan]

""


In [15]:
# Normalize the specified features in one line and save in the same columns
features_to_normalize = ['age_diagnosis_years', 'df_diet_consistency', 'df_feedings_per_day', 'total_active_hours_y', 'pa_hot_weather_months_per_year', 'pa_cold_weather_months_per_year', 'od_annual_income_range_usd']

# select features that are in data
features_to_normalize_in_data = [feature for feature in features_to_normalize if feature in data.columns]

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Normalize
if features_to_normalize_in_data:
    data[features_to_normalize_in_data] = scaler.fit_transform(data[features_to_normalize_in_data])
    print(data[features_to_normalize_in_data])

        age_diagnosis_years
0                      0.00
1                      0.04
2                      0.08
3                      0.12
4                      0.16
...                     ...
306423                 0.12
306424                 0.16
306425                 0.20
306426                 0.24
306427                 0.28

[306428 rows x 1 columns]


In [16]:
data

,dog_id,age_diagnosis_years,condition_type_Bone/Orthopedic,condition_type_Brain/Neurologic,condition_type_Cardiac,condition_type_Ear/Nose/Throat,condition_type_Endocrine,condition_type_Eye,condition_type_Gastrointestinal,condition_type_Hematopoietic,...,breed_group_Mixed Lab and Golden,breed_group_Mixed Large,breed_group_Mixed Medium,breed_group_Mixed Other,breed_group_Mixed Small,breed_group_Shepherd,breed_group_Spaniels,breed_group_Terriers,breed_group_Toy - Other,breed_group_Working dogs - Non-sport
0,100001,0.00,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,100001,0.04,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,100001,0.08,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,100001,0.12,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,100001,0.16,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306423,77040,0.12,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
306424,77040,0.16,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
306425,77040,0.20,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
306426,77040,0.24,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
# exp1 = [feat_breed_group]
# exp2 = [feat_breed_top50_v2]
# exp3 = [feat_age]
# exp4 = [feat_sex,feat_breed_type]
# exp5 = [feat_sub_breed]
# exp6 = [feat_breed_pure_or_mix]
# exp7 = [feat_breed_top50_v2,feat_breed_group]
# exp8 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex]
# exp9 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_pa_surface,feat_pa_total_hours]
# exp10 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_HotWheater,feat_residence]
# exp11 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income]
# exp12 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours]
# exp13 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours]
# exp14 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours,feat_age,feat_sex,feat_diet]
# exp15 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours,feat_age,feat_sex,feat_diet]
# exp16 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours,feat_age,feat_sex,feat_breed_pure_or_mix,feat_breed_type,feat_sub_breed,feat_diet]

In [18]:
# Separate features and labels
X = data.drop(['dog_id'] + ['condition_type_' + condition_type for condition_type in [
    'Eye', 'Ear/Nose/Throat', 'Mouth/Dental/Oral', 'Skin', 'Cardiac', 'Respiratory',
    'Gastrointestinal', 'Liver/Pancreas', 'Kidney/Urinary', 'Reproductive', 'Bone/Orthopedic',
    'Brain/Neurologic', 'Endocrine', 'Hematopoietic', 'Other Congenital Disorder',
    'Infection/Parasites', 'Toxin Consumption', 'Trauma', 'Immune-mediated', 'cancer'
]], axis=1)

y_columns = ['condition_type_' + condition_type for condition_type in [
    'Eye', 'Ear/Nose/Throat', 'Mouth/Dental/Oral', 'Skin', 'Cardiac', 'Respiratory',
    'Gastrointestinal', 'Liver/Pancreas', 'Kidney/Urinary', 'Reproductive', 'Bone/Orthopedic',
    'Brain/Neurologic', 'Endocrine', 'Hematopoietic', 'Other Congenital Disorder',
    'Infection/Parasites', 'Toxin Consumption', 'Trauma', 'Immune-mediated', 'cancer'
]]
y = data[y_columns]

# Convert y to a binary format
y_binary = (y == 1)

In [20]:
# normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

num_features = X.shape[1]
if num_features <= 100:
        hidden_layer_sizes = (60,)
elif num_features <= 200:
        hidden_layer_sizes = (60, 40)
else:
    hidden_layer_sizes = (80, 60, 40)

# Initialize the MLP model
mlp_model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=1000, alpha=0.01, early_stopping=True, random_state=42)
# Wrap the model with OneVsRestClassifier
ovr_classifier = OneVsRestClassifier(mlp_model)

# Initialize MultilabelStratifiedKFold
n_splits = 5
ml_stratified_kfold = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
# auc_scores_per_fold = []

# Initialize the dictionary that stores the results
dataset_results = {'train': [], 'validation': []}

# auc_scores_per_condition = {condition: [] for condition in y_columns}
# sample_counts_per_condition = {condition: [] for condition in y_columns}

# Iterate through the splits
for fold, (train_index, val_index) in enumerate(ml_stratified_kfold.split(X_scaled, y_binary)):
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_binary.iloc[train_index], y_binary.iloc[val_index]

    # Train the model
    ovr_classifier.fit(X_train, y_train)

    # Make predictions on the validation and train set
    y_train_pred_proba = ovr_classifier.predict_proba(X_train)
    y_val_pred_proba = ovr_classifier.predict_proba(X_val)

    # Calculate the AUC score for each disease
    train_auc_scores = [roc_auc_score(y_train.iloc[:, i], y_train_pred_proba[:, i]) for i in range(y_train.shape[1])]
    val_auc_scores = [roc_auc_score(y_val[column], y_val_pred_proba[:, i]) for i, column in enumerate(y_val.columns)]
    # print(f"\nFold {fold+1} AUC Scores for Diseases:")
    # for i, auc_score in enumerate(auc_scores):
    #     condition = y_val.columns[i]
    #     print(f"{condition}: {auc_score}")
        # auc_scores_per_condition[condition].append(auc_score)
        # sample_count = y_val[condition].sum()  # Calculate the count of positive samples for the condition
        # sample_counts_per_condition[condition].append(sample_count)

    dataset_results['train'].append(np.mean(train_auc_scores))
    dataset_results['validation'].append(np.mean(val_auc_scores))
    # auc_scores_per_fold.append(np.mean(auc_scores))

# Calculate the average AUC score for the entire experiment
# average_auc_score = np.mean(auc_scores_per_fold)
# results[exp_name] = average_auc_score
average_auc_train = np.mean(dataset_results['train'])
average_auc_validation = np.mean(dataset_results['validation'])
# results[exp_name] = {'train': average_auc_train, 'validation': average_auc_validation}

# print results for train and validation
# print(f"{exp_name}:")
print(f"    Average AUC on Train = {average_auc_train:.4f}")
print(f"    Average AUC on Validation = {average_auc_validation:.4f}")

    Average AUC on Train = 0.6555
    Average AUC on Validation = 0.6272


Save as .joblib

In [21]:
model_filepath = '../models/multi_layered_perceptron_with_breedtop50.joblib'
joblib.dump(ovr_classifier, model_filepath)

['../models/multi_layered_perceptron_with_breedtop50.joblib']

test model

In [31]:
loaded_ovr_classifier = joblib.load(model_filepath)

In [32]:
# Select a random row (Datapoint) from X_val
random_index = np.random.randint(0, X_val.shape[0])
random_data_point = X_val[random_index]

In [33]:
random_data_point

array([ 0.51868702, -0.20744848, -0.16208228, -0.17551926, -0.1969921 ,
       -0.09304337, -0.19632971, -0.06947608, -0.11741422, -0.22702257,
       -0.10087946, -0.16819269, -0.09752472, -0.09313254, -0.08066523,
       -0.10314927, -0.2294047 , -0.11659741, -0.1448359 , -0.08585156,
       -0.1809381 , -0.0961859 , -0.09677168, -0.07471309, -0.2584307 ,
       -0.09684037, -0.29070421, -0.09590904, -0.10238718, -0.09874505,
       -0.09173197, -0.15819135,  2.2353152 , -0.12377884, -0.09109675,
       -0.12688152, -0.07499964, -0.56038938, -0.10043286, -0.11219084,
       -0.275759  , -0.14106813, -0.13321344, -0.11641036, -0.07977933,
       -0.10761327, -0.09252455, -0.15686415, -0.1518557 , -0.09470617,
       -0.14318314, -0.24121457, -0.32174108, -0.19361551, -0.15702788,
       -0.25628273, -0.20108561, -0.22995585,  3.7676623 , -0.37845769,
       -0.011426  , -0.00989504, -0.05289633, -0.09936655, -0.29243691,
       -0.1594225 , -0.23872764, -0.2649725 , -0.31870595, -0.31

In [34]:
# Reshape the data point to a 2D array
random_data_point_reshaped = random_data_point.reshape(1, -1)

# Make predictions on the reshaped data point using the loaded model
new_data_predictions_proba = loaded_ovr_classifier.predict_proba(random_data_point_reshaped)
new_data_predictions_proba

array([[0.01458738, 0.02170091, 0.03777133, 0.04377064, 0.00396317,
        0.00701809, 0.01846095, 0.0027299 , 0.01580739, 0.00199349,
        0.03010136, 0.00818204, 0.00504219, 0.00106259, 0.00143441,
        0.0154293 , 0.0103628 , 0.03318697, 0.00114655, 0.00847432]])